In [7]:
import torch
import torch.nn as nn
from torch.nn import functional as F

device = 'mps' if torch.backends.mps.is_available() else 'cpu'

block_size = 8
batch_size = 4
max_iters = 10000
learning_rate = 3e-4
eval_iters = 250

print(device)

cuda


In [8]:
# Creates the sorted list of characters and vocabulary size
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
vocab_size = len(chars)

print(vocab_size)

81


In [9]:
# Initialize dictionaries for characters and their associated tokens
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }

# Define lambda functions to convert characters to tokens and vice versa
encode = lambda s : [string_to_int[c] for c in s]
decode = lambda l : ''.join([int_to_string[i] for i in l])

# Creates a Tensor of int-64s out of the tokenized characters
data = torch.tensor(encode(text), dtype=torch.long)

print(data)

tensor([80, 28, 39,  ..., 67, 57,  0])


In [11]:
# Initializes the training data and validation data
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

In [36]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [12]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range (max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1, 1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


iKczdM'ZVA8'LDSc
m]b3kKlJ:w.Fbzvuph6yMzbEPyk)c)pkZ!zW"Wy!nrx."IJ29yxYe)Z﻿Oq*Xt2:PbfSg:MJL:﻿3zm5x,(?g:BzwKZ5Mry[2:dI;Pp.-KGDu.IJ1tuM&?t]4Q
":ScAss_]qt]: &vIOqJL:ip:POL[HkCpF&3A]lLVfu)zH﻿LuOO_0m:qh:i3Wu9hr
45BJ HjwaEbS 3'0bE,0]3zbHLT&7:(eTWr6"6ASgzCxepwK9[OFJM9e!?tCq]5UB&g4Qk_51Y5xsaje& iB'7y!pudg5BzqhOa1wn80Wi42sD2P"]e*
asOv)4Q
mfcDKNwrQkDR[xbEF":TW3qhN xC2iB7!jegD?9zfLW(s z-8eI&;pCpHtX﻿a9[[s7HzV6L&dAavxOCdbJh5a*WO(K"CT"YN!g'HW7-'!A*EFToNJ_DJk]ODVeA)I."a&AhsR F0m﻿[Kg]Z3kBa7KRWkB6I&J1wXEF_m&&.s_0e


In [43]:
# Create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f'Step: {iter} | Train Loss: {losses['train']:.4f} | Val Loss: {losses['val']:.4f}')
    
    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

Step: 0 | Train Loss: 2.6290 | Val Loss: 2.6892
Step: 250 | Train Loss: 2.6453 | Val Loss: 2.6466
Step: 500 | Train Loss: 2.6159 | Val Loss: 2.6742
Step: 750 | Train Loss: 2.6384 | Val Loss: 2.6663
Step: 1000 | Train Loss: 2.6063 | Val Loss: 2.6477
Step: 1250 | Train Loss: 2.6198 | Val Loss: 2.6477
Step: 1500 | Train Loss: 2.6239 | Val Loss: 2.6716
Step: 1750 | Train Loss: 2.6048 | Val Loss: 2.6341
Step: 2000 | Train Loss: 2.6238 | Val Loss: 2.6238
Step: 2250 | Train Loss: 2.6003 | Val Loss: 2.6143
Step: 2500 | Train Loss: 2.6124 | Val Loss: 2.6376
Step: 2750 | Train Loss: 2.5919 | Val Loss: 2.6475
Step: 3000 | Train Loss: 2.5808 | Val Loss: 2.6433
Step: 3250 | Train Loss: 2.5825 | Val Loss: 2.6342
Step: 3500 | Train Loss: 2.5823 | Val Loss: 2.6145
Step: 3750 | Train Loss: 2.5686 | Val Loss: 2.5938
Step: 4000 | Train Loss: 2.5831 | Val Loss: 2.5951
Step: 4250 | Train Loss: 2.5379 | Val Loss: 2.5984
Step: 4500 | Train Loss: 2.5738 | Val Loss: 2.5723
Step: 4750 | Train Loss: 2.5576 | Val

In [13]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


oH6p()jcJ1w,r"gzb?lL&(x!7R55UMkssKQzC:﻿_o0!7L'7M.T"1xRE&gi51FD!X﻿I8y;T"z9"0BaeAGD
ZzU*!.
tu8i*08wzseLzqq9p
p,Pw"!fi4EP4!"zHW_y!2CEKcx3f3'c4Xh3fS75i(2j"CzCTxdf(ugl﻿oZ2v]7﻿a'kI8BJeIfPGL4znp1bT!O!﻿4Q;pi3mh_8Be!]up,8k'eJ2&.*wRgBa5xYTQUYore*fL't5
8i('xOCa&xOYLMl[S5x3(X;3PqAN!x6,Si(97WcUB_R66l19Ur:()FY2dJBYwW.﻿﻿JwCYy!akNpV6npE8t-p:RqhgQ5MxeI
e[rQ(GDgFb2sy'w]BD;XyDUkvoS'xR2'H_07JLn*H-rXj2vJ7y!?u6xO
;;vSKRtX7dAZA X9)j&x3fP7fS7XX8JHkvQbZpZ2c&;rprml*ob?x78hr &i()C'Kqht7"L[1sW_Nas_PLbC 6)74p1V2Yy!._NKclh;2
